# Coronavirus Data Analysis

In [2]:
import pandas as pd
import numpy as np

In [3]:
spark.sparkContext.applicationId

'application_1636778915296_0002'

In [4]:
pd.options.display.max_columns = 35

In [5]:
spark.stop()

spark = SparkSession.builder \
                    .config('spark.dynamicAllocation.maxExecutors', 50) \
                    .config('spark.executor.cores', '3') \
                    .config('spark.executor.memory', '15g') \
                    .config('spark.driver.memory', '40g') \
                    .config('spark.python.worker.memory', '2g') \
                    .config('spark.sql.shuffle.partitions', '2000') \
                    .config('spark.network.timeout', '2000') \
                    .config('spark.sql.broadcastTimeout', '2000') \
                    .config('spark.driver.maxResultSize', '6g') \
                    .config('mapreduce.fileoutputcommitter.algorithm.version', '2') \
                    .getOrCreate()
            
spark.sparkContext.applicationId

'application_1636778915296_0003'

In [6]:
from foresight.discern import broadcast_discern, push_discern, pop_discern

### care management new default will be 5e259fd5-75b5-4d49-82d3-2d4e92dca831 for any new clients or 
###  any algorithm wiki not line for line with Cerner Standard then would be using the new context  

pd.set_option('display.max_colwidth', None)

# Location of the discernontology database
root = 's3://consult-datalab-persistence-s3-data/discernontology/v1/'   ##<------------- change it to your stack without the v1/

contextid = ['da0acee6-1e2e-4384-9fc7-a58c259d0c50', '5E259FD575B54D4982D32D4E92DCA831', '6E70BFD2F17848BBB19A31ECDFB49EE7'] 

# Loop that removes all hypthens and captializes all letters. Sets up the file location
for contextid in contextid:
    contextid = contextid.upper().replace("-", "")
    push_discern(spark, contextid, discern_root=root)
    print(contextid)

defaultcontext = 'DA0ACEE61E2E43849FC7A58C259D0C50'   
defaultcontext1 = '5E259FD575B54D4982D32D4E92DCA831' 

DA0ACEE61E2E43849FC7A58C259D0C50
5E259FD575B54D4982D32D4E92DCA831
6E70BFD2F17848BBB19A31ECDFB49EE7


In [7]:
db = "real_world_data_2021_Q2"
spark.sql("USE {}".format(db))
print("Using database: {},".format(db))

Using database: real_world_data_2021_Q2,


In [8]:
spark.sql('show tables').toPandas()

,database,tableName,isTemporary
0,real_world_data_2021_q2,allergy,False
1,real_world_data_2021_q2,clinical_event,False
2,real_world_data_2021_q2,condition,False
3,real_world_data_2021_q2,demographics,False
4,real_world_data_2021_q2,encounter,False
5,real_world_data_2021_q2,immunization,False
6,real_world_data_2021_q2,lab,False
7,real_world_data_2021_q2,measurement,False
8,real_world_data_2021_q2,medication,False
9,real_world_data_2021_q2,medication_administration,False


### Create Variables for Queries

In [9]:
# Look back days variables
daysInterval = 365
print(daysInterval)
maxDays = 365
print(maxDays)
oneYearInterval=365
print(oneYearInterval)

# Username variable
user_id = 'rc047407'           ####<-------------------- change to your user id (Ex: tl068507)
userName = 'user_' + user_id
print(userName)

# Algorithm name variable
algorithmName='CoronavirusLengthOfStay_HighLevel'
print(algorithmName)

365
365
365
user_rc047407
CoronavirusLengthOfStay_HighLevel


In [10]:
spark.sql("""CREATE SCHEMA IF NOT EXISTS {userName}""".format(**{
    'userName':userName
})).show()

++
||
++
++



In [11]:
spark.sql(f"""select * FROM {userName}.{algorithmName}_Hospital h  """).printSchema()

root
 |-- tenant: integer (nullable = true)
 |-- tenant_bed_size: string (nullable = true)
 |-- tenant_speciality: string (nullable = true)
 |-- tenant_segment: string (nullable = true)
 |-- tenant_zip_code: string (nullable = true)
 |-- personId: string (nullable = true)
 |-- encounterId: string (nullable = true)
 |-- serviceDate: string (nullable = true)
 |-- dischargeDate: string (nullable = true)
 |-- length_of_stay: integer (nullable = true)
 |-- zip_code: string (nullable = true)
 |-- deceased: integer (nullable = true)
 |-- discharged_to_skilled_nursing_facility: integer (nullable = true)
 |-- discharged_to_hospice: integer (nullable = true)
 |-- died_in_the_hospital: integer (nullable = true)
 |-- discharged_to_rehab_facility: integer (nullable = true)
 |-- discharged_to_long_term_care_facility: integer (nullable = true)
 |-- intensive_care_visit: integer (nullable = true)
 |-- intensive_care_medical_service: integer (nullable = true)
 |-- palliative_care_visit: integer (nullab

In [23]:
spark.sql("""select * from measurement """).printSchema()

root
 |-- measurementid: string (nullable = true)
 |-- encounterid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- measurementcode: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- loincclass: string (nullable = true)
 |-- type: string (nullable = true)
 |-- servicedate: string (nullable = true)
 |-- serviceperiod: struct (nullable = true)
 |    |-- startDate: string (nullable = true)
 |    |-- endDate: string (nullable = true)
 |-- typedvalue: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |   

In [11]:
spark.sql("""select * from medication """).printSchema()

root
 |-- medicationid: string (nullable = true)
 |-- encounterid: string (nullable = true)
 |-- personid: string (nullable = true)
 |-- intendeddispenser: string (nullable = true)
 |-- startdate: string (nullable = true)
 |-- intendedadministrator: string (nullable = true)
 |-- doseunit: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (nullable = true)
 |    |    |-- primaryDisplay: string (nullable = true)
 |    |-- standardCodings: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- id: string (nullable = true)
 |    |    |    |-- codingSystemId: string (nullable = true)
 |    |    |    |-- primaryDisplay: string (nullable = true)
 |-- stopdate: string (nullable = true)
 |-- category: struct (nullable = true)
 |    |-- standard: struct (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- codingSystemId: string (n

In [12]:
spark.sql(f"""


    SELECT distinct 
        h.tenant
        ,count(distinct h.personId)
        
            FROM {userName}.{algorithmName}_Hospital h  
            
  group by 1          

  order by 1
    


""").toPandas()

,tenant,count(DISTINCT personId)
0,1,1862
1,2,30
2,3,1340
3,4,18
4,5,130
...,...,...
91,98,10
92,99,255
93,100,205
94,101,471


In [13]:
spark.sql(f"""
select distinct l.type
        
     FROM {db}.lab l
         WHERE NOT HAS_ANY_CONCEPT_IN_CONTEXT(l.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                
""").toPandas()

,type
0,TEXT
1,DATE
2,NUMERIC
3,CODIFIED


In [14]:
commonmedicationsDf = spark.sql(f"""

select --drugcode_names[0] as drugcode_name
      drugcode_name
      ,count(*) as count
      
from (      

--   select split(lower(m.drugcode.standard.primaryDisplay), ' ') as drugcode_names
     select lower(m.drugcode.standard.primaryDisplay) as drugcode_name
   --      ,count(distinct h.personId) as count
       


  
        FROM {userName}.{algorithmName}_Hospital h  
        

       

        JOIN {db}.medication m
            ON  h.encounterId = m.encounterId
                and h.personId = m.personID
                and h.tenant = m.tenant
 
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(m.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
           
           
          where m.drugcode.standard.primaryDisplay is not null    
          
    
          /*      and  ( lower(m.drugcode.standard.primaryDisplay) like '%dexamethasone%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%remdesivir%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%veklury%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%monoclonal%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%antibodies%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%mabs%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%dexamethasone%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%convalescent%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%plasma%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%bamlanivimab%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%ly-cov555%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%etesevimab%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%casirivimab%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%imdevimab%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%regn-cov2%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%hydroxychloroquine%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%chloroquine%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%azithromycin%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%tocilizumab%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%actemra%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%il-6%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%acalabrutinib%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%kevzara%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%sarilumab%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%calquence%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%baricitinib%'
                     or lower(m.drugcode.standard.primaryDisplay) like '%olumiant%'
                     )  */
  ) drugs

  
         group by 1
         
         having count >= 50000  --100000
     /*     or  ( lower(drugcode_name) like '%dexamethasone%'
                     or lower(drugcode_name) like '%remdesivir%'
                     or lower(drugcode_name) like '%veklury%'
                     or lower(drugcode_name) like '%monoclonal%'
                     or lower(drugcode_name) like '%antibodies%'
                     or lower(drugcode_name) like '%mabs%'
                     or lower(drugcode_name) like '%dexamethasone%'
                     or lower(drugcode_name) like '%convalescent%'
                     or lower(drugcode_name) like '%plasma%'
                     or lower(drugcode_name) like '%bamlanivimab%'
                     or lower(drugcode_name) like '%ly-cov555%'
                     or lower(drugcode_name) like '%etesevimab%'
                     or lower(drugcode_name) like '%casirivimab%'
                     or lower(drugcode_name) like '%imdevimab%'
                     or lower(drugcode_name) like '%regn-cov2%'
                     or lower(drugcode_name) like '%hydroxychloroquine%'
                     or lower(drugcode_name) like '%chloroquine%'
                     or lower(drugcode_name) like '%azithromycin%'
                     or lower(drugcode_name) like '%tocilizumab%'
                     or lower(drugcode_name) like '%actemra%'
                     or lower(drugcode_name) like '%il-6%'
                     or lower(drugcode_name) like '%acalabrutinib%'
                     or lower(drugcode_name) like '%kevzara%'
                     or lower(drugcode_name) like '%sarilumab%'
                     or lower(drugcode_name) like '%calquence%'
                     or lower(drugcode_name) like '%baricitinib%'
                     or lower(drugcode_name) like '%olumiant%'
                     )  */
          
         order by 2 desc 
         
      
         
   
  
         
""")

In [15]:
commonmedicationsDf.toPandas()

,drugcode_name,count
0,sodium chloride 0.9% intravenous solution,1336830
1,propofol,832049
2,fentanyl,568309
3,insulin lispro,529061
4,midazolam,320867
...,...,...
57,doxycycline,54379
58,miralax,53466
59,amlodipine,53320
60,vitamin c,53058


In [16]:
commonmedicationsDf.toPandas().head(25)

,drugcode_name,count
0,sodium chloride 0.9% intravenous solution,1336830
1,propofol,832049
2,fentanyl,568309
3,insulin lispro,529061
4,midazolam,320867
5,ceftriaxone,309214
6,potassium chloride,290269
7,dexamethasone,279358
8,heparin,246381
9,dexmedetomidine,236389


In [18]:
commonmedicationsDf.toPandas().to_excel("test3.xlsx")

In [12]:
spark.sql(f"""
select distinct
         m.drugcode.standard.primaryDisplay as drugcode_display
 
        FROM {userName}.{algorithmName}_Hospital h  
        

       

        JOIN {db}.medication m
            ON  h.encounterId = m.encounterId
                and h.personId = m.personID
                and h.tenant = m.tenant
 
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(m.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                          
          
    
                and  ( lower(m.drugcode.standard.primaryDisplay) like '%convalescent%'
                       or lower(m.drugcode.standard.primaryDisplay) like '%bamlanivimib%'
                       or lower(m.drugcode.standard.primaryDisplay) like '%serine%'
                       or lower(m.drugcode.standard.primaryDisplay) like '%umifenovir%'
                       or lower(m.drugcode.standard.primaryDisplay) like '%fluvoxamine%' )
           
        order by 1
        
""").toPandas()

,drugcode_display
0,fluvoxaMINE
1,"fluvoxaMINE 100 mg oral capsule, extended release"
2,fluvoxaMINE 100 mg oral tablet
3,"fluvoxaMINE 150 mg oral capsule, extended release"
4,fluvoxaMINE 25 mg oral tablet
5,fluvoxaMINE 50 mg oral tablet
6,fluvoxaMINE extended release


In [13]:
medicationsDf = spark.sql(f"""
   select 
          h.tenant
         ,h.personId
         ,h.encounterId
         ,m.drugcode.standard.id as drugcode_id
         ,m.drugcode.standard.primaryDisplay as drugcode_display
         ,m.drugcode.standard.codingSystemId as drugcode_system_id
         ,m.startdate
         ,m.stopdate
         ,m.doseunit.standard.id as doseunit_id
         ,m.doseunit.standard.primaryDisplay as doseunit_display
         ,m.doseunit.standard.codingSystemId as doseunit_system_id
         ,m.route.standard.id as route_id
         ,m.route.standard.primaryDisplay as route_display
         ,m.route.standard.codingSystemId as route_system_id
  
        FROM {userName}.{algorithmName}_Hospital h  
        

       

        JOIN {db}.medication m
            ON  h.encounterId = m.encounterId
                and h.personId = m.personID
                and h.tenant = m.tenant
 
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(m.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                          
          
    
                and  ( lower(m.drugcode.standard.primaryDisplay) in ('sodium chloride 0.9% intravenous solution',
                        'propofol',
                        'fentanyl',
                        'insulin lispro',
                        'midazolam',
                        'ceftriaxone',
                        'potassium chloride',
                        'dexamethasone',
                        'heparin',
                        'dexmedetomidine',
                        'tylenol',
                        'cefepime',
                        'acetaminophen',
                        'azithromycin',
                        'piperacillin-tazobactam',
                        'lasix',
                        'insulin aspart',
                        'remdesivir',
                        'norepinephrine',
                        'vancomycin',
                        'lovenox',
                        'morphine',
                        'zofran',
                        'magnesium sulfate',
                        'dextrose 5% in water intravenous solution',
                        'lactated ringers injection intravenous solution',
                        'glucose',
                        'ondansetron',
                        'lantus',
                        'insulin regular',
                        'glucagon',
                        'aspirin',
                        'dextrose 50% in water intravenous solution',
                        'enoxaparin',
                        'pantoprazole',
                        'furosemide',
                        'hydralazine',
                        'meropenem',
                        'albuterol',
                        'ativan',
                        '0.9nacl',
                        'rocephin',
                        'lorazepam',
                        'sterile water',
                        'hydromorphone',
                        'atorvastatin',
                        'rocuronium',
                        'albuterol 90 mcg/inh inhalation aerosol',
                        'sodium chloride',
                        'protonix',
                        'insulin glargine',
                        'sodium bicarbonate',
                        'oxycodone',
                        'zinc sulfate',
                        'famotidine',
                        'amiodarone',
                        'sodium chloride 0.9% injectable solution',
                        'doxycycline',
                        'miralax',
                        'amlodipine',
                        'vitamin c',
                        'dextrose',
                        'bamlanivimab',
                        'fluvoxamine'
                             )
                     )
                               

          
         order by 1,2,3,4
         
      
         
   
  
         
""")

In [14]:
##### Drop existing table, if there was one 
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_DetailedPatientsMedications""").show() 

++
||
++
++



In [15]:
medicationsDf.write.saveAsTable(f"""{userName}.{algorithmName}_DetailedPatientsMedications """)

In [16]:
spark.sql(f"""select * from {userName}.{algorithmName}_DetailedPatientsMedications""").limit(50).toPandas()

,tenant,personId,encounterId,drugcode_id,drugcode_display,drugcode_system_id,startdate,stopdate,doseunit_id,doseunit_display,doseunit_system_id,route_id,route_display,route_system_id
0,74,7b2e2b5c-12de-45c9-802f-f5e06e0e195f,ba0ca55f-89e3-4e8f-9464-fa6bc4b1edd9,12084,LORazepam,2.16.840.1.113883.6.312,2021-01-06T07:54:00+00:00,2021-01-12T16:01:24+00:00,mg,milligram,2.16.840.1.113883.6.8,47625008,Intravenous,2.16.840.1.113883.6.96
1,74,7b2e2b5c-12de-45c9-802f-f5e06e0e195f,ba0ca55f-89e3-4e8f-9464-fa6bc4b1edd9,12136,dexamethasone,2.16.840.1.113883.6.312,2021-01-07T21:00:00+00:00,2021-01-12T16:01:26+00:00,mg,milligram,2.16.840.1.113883.6.8,47625008,Intravenous,2.16.840.1.113883.6.96
2,74,7b2e2b5c-12de-45c9-802f-f5e06e0e195f,ba0ca55f-89e3-4e8f-9464-fa6bc4b1edd9,12449,albuterol,2.16.840.1.113883.6.312,2021-01-06T15:00:00+00:00,2021-01-06T15:45:00+00:00,mg,milligram,2.16.840.1.113883.6.8,447694001,Respiratory tract route,2.16.840.1.113883.6.96
3,74,7b2e2b5c-12de-45c9-802f-f5e06e0e195f,ba0ca55f-89e3-4e8f-9464-fa6bc4b1edd9,27645,albuterol 90 mcg/inh inhalation aerosol,2.16.840.1.113883.6.312,2021-01-11T18:02:00+00:00,2021-03-12T18:02:00+00:00,ug,microgram,2.16.840.1.113883.6.8,447694001,Respiratory tract route,2.16.840.1.113883.6.96
4,74,7b2e2b5c-12de-45c9-802f-f5e06e0e195f,ba0ca55f-89e3-4e8f-9464-fa6bc4b1edd9,29754,0.9NaCl,2.16.840.1.113883.6.312,2021-01-08T06:48:00+00:00,2021-01-12T16:01:23+00:00,mL,milliliter,2.16.840.1.113883.6.8,47625008,Intravenous,2.16.840.1.113883.6.96
5,74,7b2e2b5c-12de-45c9-802f-f5e06e0e195f,ba0ca55f-89e3-4e8f-9464-fa6bc4b1edd9,5902,Sodium Chloride 0.9% intravenous solution,2.16.840.1.113883.6.311,2021-01-08T05:53:07+00:00,2021-01-08T05:53:07+00:00,None,None,None,385432009,Not applicable,2.16.840.1.113883.6.96
6,74,7b2e2b5c-12de-45c9-802f-f5e06e0e195f,ba0ca55f-89e3-4e8f-9464-fa6bc4b1edd9,5902,Sodium Chloride 0.9% intravenous solution,2.16.840.1.113883.6.311,2021-01-06T01:08:33+00:00,2021-01-06T01:08:33+00:00,None,None,None,385432009,Not applicable,2.16.840.1.113883.6.96
7,74,7b2e2b5c-12de-45c9-802f-f5e06e0e195f,ba0ca55f-89e3-4e8f-9464-fa6bc4b1edd9,5902,Sodium Chloride 0.9% intravenous solution,2.16.840.1.113883.6.311,2021-01-06T00:49:00+00:00,2021-01-06T01:58:48+00:00,mL,milliliter,2.16.840.1.113883.6.8,47625008,Intravenous,2.16.840.1.113883.6.96
8,74,7b2e2b5c-12de-45c9-802f-f5e06e0e195f,ba0ca55f-89e3-4e8f-9464-fa6bc4b1edd9,6732,Tylenol,2.16.840.1.113883.6.312,2021-01-05T20:45:00+00:00,2021-01-05T20:52:41+00:00,mg,milligram,2.16.840.1.113883.6.8,26643006,Oral route,2.16.840.1.113883.6.96
9,74,7b2e2b5c-12de-45c9-802f-f5e06e0e195f,ba0ca55f-89e3-4e8f-9464-fa6bc4b1edd9,6732,Tylenol,2.16.840.1.113883.6.312,2021-01-06T13:35:00+00:00,2021-01-12T16:01:24+00:00,mg,milligram,2.16.840.1.113883.6.8,127490009,Gastrostomy route,2.16.840.1.113883.6.96


In [19]:
##spark.sql(f"""select * from {userName}.{algorithmName}_LabSummary """).toPandas().to_excel("test.xlsx")

In [17]:
medicationsTransposeDf = spark.sql(f"""
select 
          h.tenant
         ,h.personId
         ,h.encounterId
         ,m.startdate
         ,m.stopdate
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'sodium chloride 0.9% intravenous solution', m.drugCode.standard.id, null)) as taken_sodium_chloride_09percent_intravenous_solution_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'propofol', m.drugCode.standard.id, null)) as taken_propofol_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'fentanyl', m.drugCode.standard.id, null)) as taken_fentanyl_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'insulin lispro', m.drugCode.standard.id, null)) as taken_insulin_lispro_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'midazolam', m.drugCode.standard.id, null)) as taken_midazolam_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'ceftriaxone', m.drugCode.standard.id, null)) as taken_ceftriaxone_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'potassium chloride', m.drugCode.standard.id, null)) as taken_potassium_chloride_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'dexamethasone', m.drugCode.standard.id, null)) as taken_dexamethasone_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'heparin', m.drugCode.standard.id, null)) as taken_heparin_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'dexmedetomidine', m.drugCode.standard.id, null)) as taken_dexmedetomidine_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'tylenol', m.drugCode.standard.id, null)) as taken_tylenol_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'cefepime', m.drugCode.standard.id, null)) as taken_cefepime_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'acetaminophen', m.drugCode.standard.id, null)) as taken_acetaminophen_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'azithromycin', m.drugCode.standard.id, null)) as taken_azithromycin_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'piperacillin-tazobactam', m.drugCode.standard.id, null)) as taken_piperacillin_tazobactam_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'lasix', m.drugCode.standard.id, null)) as taken_lasix_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'insulin aspart', m.drugCode.standard.id, null)) as taken_insulin_aspart_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'remdesivir', m.drugCode.standard.id, null)) as taken_remdesivir_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'norepinephrine', m.drugCode.standard.id, null)) as taken_norepinephrine_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'vancomycin', m.drugCode.standard.id, null)) as taken_vancomycin_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'lovenox', m.drugCode.standard.id, null)) as taken_lovenox_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'morphine', m.drugCode.standard.id, null)) as taken_morphine_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'zofran', m.drugCode.standard.id, null)) as taken_zofran_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'magnesium sulfate', m.drugCode.standard.id, null)) as taken_magnesium_sulfate_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'dextrose 5% in water intravenous solution', m.drugCode.standard.id, null)) as taken_dextrose_5percent_in_water_intravenous_solution_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'lactated ringers injection intravenous solution', m.drugCode.standard.id, null)) as taken_lactated_ringers_injection_intravenous_solution_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'glucose', m.drugCode.standard.id, null)) as taken_glucose_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'ondansetron', m.drugCode.standard.id, null)) as taken_ondansetron_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'lantus', m.drugCode.standard.id, null)) as taken_lantus_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'insulin regular', m.drugCode.standard.id, null)) as taken_insulin_regular_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'glucagon', m.drugCode.standard.id, null)) as taken_glucagon_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'aspirin', m.drugCode.standard.id, null)) as taken_aspirin_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'dextrose 50% in water intravenous solution', m.drugCode.standard.id, null)) as taken_dextrose_50percent_in_water_intravenous_solution_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'enoxaparin', m.drugCode.standard.id, null)) as taken_enoxaparin_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'pantoprazole', m.drugCode.standard.id, null)) as taken_pantoprazole_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'furosemide', m.drugCode.standard.id, null)) as taken_furosemide_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'hydralazine', m.drugCode.standard.id, null)) as taken_hydralazine_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'meropenem', m.drugCode.standard.id, null)) as taken_meropenem_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'albuterol', m.drugCode.standard.id, null)) as taken_albuterol_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'ativan', m.drugCode.standard.id, null)) as taken_ativan_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = '0.9nacl', m.drugCode.standard.id, null)) as taken_09nacl_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'rocephin', m.drugCode.standard.id, null)) as taken_rocephin_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'lorazepam', m.drugCode.standard.id, null)) as taken_lorazepam_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'sterile water', m.drugCode.standard.id, null)) as taken_sterile_water_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'hydromorphone', m.drugCode.standard.id, null)) as taken_hydromorphone_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'atorvastatin', m.drugCode.standard.id, null)) as taken_atorvastatin_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'rocuronium', m.drugCode.standard.id, null)) as taken_rocuronium_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'albuterol 90 mcg/inh inhalation aerosol', m.drugCode.standard.id, null)) as taken_albuterol_90_mcg_inh_inhalation_aerosol_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'sodium chloride', m.drugCode.standard.id, null)) as taken_sodium_chloride_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'protonix', m.drugCode.standard.id, null)) as taken_protonix_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'insulin glargine', m.drugCode.standard.id, null)) as taken_insulin_glargine_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'sodium bicarbonate', m.drugCode.standard.id, null)) as taken_sodium_bicarbonate_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'oxycodone', m.drugCode.standard.id, null)) as taken_oxycodone_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'zinc sulfate', m.drugCode.standard.id, null)) as taken_zinc_sulfate_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'famotidine', m.drugCode.standard.id, null)) as taken_famotidine_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'amiodarone', m.drugCode.standard.id, null)) as taken_amiodarone_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'sodium chloride 0.9% injectable solution', m.drugCode.standard.id, null)) as taken_sodium_chloride_09percent_injectable_solution_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'doxycycline', m.drugCode.standard.id, null)) as taken_doxycycline_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'miralax', m.drugCode.standard.id, null)) as taken_miralax_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'amlodipine', m.drugCode.standard.id, null)) as taken_amlodipine_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'vitamin c', m.drugCode.standard.id, null)) as taken_vitamin_c_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'dextrose', m.drugCode.standard.id, null)) as taken_dextrose_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'bamlanivimab', m.drugCode.standard.id, null)) as taken_bamlanivimab_drugcode
,MAX(IF(LOWER(m.drugcode.standard.primaryDisplay) = 'fluvoxamine', m.drugCode.standard.id, null)) as taken_fluvoxamine_drugcode
  
        FROM {userName}.{algorithmName}_Hospital h  
        

       

        JOIN {db}.medication m
            ON  h.encounterId = m.encounterId
                and h.personId = m.personID
                and h.tenant = m.tenant
 
                AND NOT HAS_ANY_CONCEPT_IN_CONTEXT(m.status, array('ENTERED_IN_ERROR_PROBSTAT', 'ERROR_ENTRY_DELETED_QUAL', 
                                                       'CANCELED_PROBSTAT', 'CANCELED_QUAL', 'DEFERRED_QUAL',
                                                       'UNAUTHORIZED_QUAL', 'VOIDED_QUAL', 'REJECTED_QUAL'), '5E259FD575B54D4982D32D4E92DCA831')  
                
                          
          
    
                and  ( lower(m.drugcode.standard.primaryDisplay) in ('sodium chloride 0.9% intravenous solution',
                                'propofol',
                                'fentanyl',
                                'insulin lispro',
                                'midazolam',
                                'ceftriaxone',
                                'potassium chloride',
                                'dexamethasone',
                                'heparin',
                                'dexmedetomidine',
                                'tylenol',
                                'cefepime',
                                'acetaminophen',
                                'azithromycin',
                                'piperacillin-tazobactam',
                                'lasix',
                                'insulin aspart',
                                'remdesivir',
                                'norepinephrine',
                                'vancomycin',
                                'lovenox',
                                'morphine',
                                'zofran',
                                'magnesium sulfate',
                                'dextrose 5% in water intravenous solution',
                                'lactated ringers injection intravenous solution',
                                'glucose',
                                'ondansetron',
                                'lantus',
                                'insulin regular',
                                'glucagon',
                                'aspirin',
                                'dextrose 50% in water intravenous solution',
                                'enoxaparin',
                                'pantoprazole',
                                'furosemide',
                                'hydralazine',
                                'meropenem',
                                'albuterol',
                                'ativan',
                                '0.9nacl',
                                'rocephin',
                                'lorazepam',
                                'sterile water',
                                'hydromorphone',
                                'atorvastatin',
                                'rocuronium',
                                'albuterol 90 mcg/inh inhalation aerosol',
                                'sodium chloride',
                                'protonix',
                                'insulin glargine',
                                'sodium bicarbonate',
                                'oxycodone',
                                'zinc sulfate',
                                'famotidine',
                                'amiodarone',
                                'sodium chloride 0.9% injectable solution',
                                'doxycycline',
                                'miralax',
                                'amlodipine',
                                'vitamin c',
                                'dextrose',
                                'bamlanivimab',
                                'fluvoxamine'
                             )
                     )
                               

         group by 1,2,3,4,5
         order by 1,2,3,4
         
      
         
   
         
""")

In [18]:
medicationsTransposeDf.limit(50).toPandas()

,tenant,personId,encounterId,startdate,stopdate,taken_sodium_chloride_09percent_intravenous_solution_drugcode,taken_propofol_drugcode,taken_fentanyl_drugcode,taken_insulin_lispro_drugcode,taken_midazolam_drugcode,taken_ceftriaxone_drugcode,taken_potassium_chloride_drugcode,taken_dexamethasone_drugcode,taken_heparin_drugcode,taken_dexmedetomidine_drugcode,taken_tylenol_drugcode,taken_cefepime_drugcode,...,taken_albuterol_90_mcg_inh_inhalation_aerosol_drugcode,taken_sodium_chloride_drugcode,taken_protonix_drugcode,taken_insulin_glargine_drugcode,taken_sodium_bicarbonate_drugcode,taken_oxycodone_drugcode,taken_zinc_sulfate_drugcode,taken_famotidine_drugcode,taken_amiodarone_drugcode,taken_sodium_chloride_09percent_injectable_solution_drugcode,taken_doxycycline_drugcode,taken_miralax_drugcode,taken_amlodipine_drugcode,taken_vitamin_c_drugcode,taken_dextrose_drugcode,taken_bamlanivimab_drugcode,taken_fluvoxamine_drugcode
0,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T01:54:04+00:00,2021-02-11T01:54:04+00:00,None,None,None,None,None,None,None,None,d00252,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T01:54:13+00:00,2021-02-11T01:54:13+00:00,None,None,None,None,None,None,None,None,d00252,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T02:00:00+00:00,2021-02-11T02:08:17+00:00,None,None,None,None,None,None,None,None,12175,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T04:28:00+00:00,2021-02-12T18:12:00+00:00,None,None,None,None,None,None,None,None,None,None,6732,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T04:28:00+00:00,2021-02-15T21:40:23+00:00,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-11T18:24:35+00:00,2021-02-11T18:24:35+00:00,None,None,None,None,None,None,None,None,d00252,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-12T10:10:28+00:00,2021-02-12T10:10:28+00:00,None,None,None,None,None,None,None,None,d00252,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-12T19:00:00+00:00,2021-02-15T21:40:26+00:00,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-13T16:24:00+00:00,2021-02-15T21:40:23+00:00,None,None,None,None,None,None,None,12136,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,1,00156fe9-b997-410a-b95f-c186433d27ab,e7e78e3a-2792-4001-b05e-b968ce707bf5,2021-02-13T17:00:00+00:00,2021-02-13T18:30:40+00:00,None,None,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [19]:
# Drop existing table, if there was one 
spark.sql(f"""DROP TABLE IF EXISTS {userName}.{algorithmName}_SummaryPatientsMedications""").show() 

++
||
++
++



In [20]:
medicationsTransposeDf.write.saveAsTable(f"""{userName}.{algorithmName}_SummaryPatientsMedications """)